In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

import clickhouse_connect

# Show all rows
pd.set_option("display.max_rows", None)

# Show all columns
pd.set_option("display.max_columns", None)

# Make sure wide DataFrames don't wrap
pd.set_option("display.width", None)

# Show the full content of each column (no '...')
pd.set_option("display.max_colwidth", None)


In [4]:
client = clickhouse_connect.get_client(
    host='yaujulxk39.eastus2.azure.clickhouse.cloud',      # or server IP
    port=8443,             # HTTP port (default)
    username='default',
    password='~gZjRLjjOJh1i',
    database='Competitive_Analysis'
)

In [5]:
# ---- Query ----
df_hum = client.query_df("""
    SELECT *
    FROM Competitive_Analysis.DRGHistoricalReviewInformation
    WHERE Client = 'Hum'
""")

"""# ---- Metrics ----
log_df = pd.DataFrame([{
    "Run_Timestamp": datetime.now(),
    "Max_InitialDeterminationDate": df_hum['InitialDeterminationDate'].max(),
    "Total_Claims": len(df_hum),
    "Approved_Claims": (df_hum['InitialDeterminationStatus'] == 'Approved').sum(),
    "Denied_Claims": (df_hum['InitialDeterminationStatus'] == 'Denied').sum(),
    "Total_IDSavings": df_hum['IDSavings'].sum()
}])

# ---- File info ----
log_file = r"C:\Arun-MIX\DRG Analysis Report\DRG_DB_Query_Log.xlsx"
sheet_name = "Humana"

# ---- Append safely ----
existing_df = pd.read_excel(log_file, sheet_name=sheet_name)
final_df = pd.concat([existing_df, log_df], ignore_index=True)

with pd.ExcelWriter(
    log_file,
    engine="openpyxl",
    mode="a",
    if_sheet_exists="replace"
) as writer:
    final_df.to_excel(writer, sheet_name=sheet_name, index=False)

print("✅ Metrics successfully appended to Excel log (Humana sheet)")"""
df_hum.shape

(73699, 291)

In [6]:
# -------------------------------------------------
# Initial shape of the dataset
# -------------------------------------------------
print(df_hum.shape)  

df_hum = df_hum[df_hum['InitialDenialType'] == 'Substantive']
print(df_hum.shape) 

# -------------------------------------------------
# Identify valid Group_Type values
# -------------------------------------------------

group_type_unique = df_hum['Group_Type'].dropna().unique()
print(group_type_unique)  

# -------------------------------------------------
# Filter dataset to only valid Group_Type rows
# Use .copy() to avoid SettingWithCopyWarning
# -------------------------------------------------
df_hum_group_type = df_hum[df_hum['Group_Type'].isin(group_type_unique)].copy()
print(df_hum_group_type.shape)  

# -------------------------------------------------
# Calculate ADRG length
# -------------------------------------------------
df_hum_group_type.loc[:, 'len_adrg'] = (
    df_hum_group_type['ADRG'].astype(str).str.len()
)

print(df_hum_group_type['len_adrg'].value_counts())
print(df_hum_group_type['Group_Type'].value_counts())

# -------------------------------------------------
# Remove invalid DRG / Group_Type combinations
#   1) Group_Type = '01' and ADRG length = 4
#   2) Group_Type = '10' and ADRG length = 3
# -------------------------------------------------
mask_remove = (
    ((df_hum_group_type['Group_Type'] == '01') & (df_hum_group_type['len_adrg'] == 4)) |
    ((df_hum_group_type['Group_Type'] == '10') & (df_hum_group_type['len_adrg'] == 3))
)

df_hum_drg_rem = df_hum_group_type.loc[~mask_remove].copy()
print(df_hum_drg_rem.shape)

# -------------------------------------------------
# Create a copy of InitialDeterminationDate
# -------------------------------------------------
df_hum_drg_rem.loc[:, 'InitialDeterminationDate_copy'] = (
    df_hum_drg_rem['InitialDeterminationDate']
)

print(
    df_hum_drg_rem['Control_ID']
    .value_counts()
    .sort_values(ascending=False)
    .head()
)

# -------------------------------------------------
# Convert date column to datetime
# -------------------------------------------------
df_hum_drg_rem.loc[:, 'InitialDeterminationDate_copy'] = pd.to_datetime(
    df_hum_drg_rem['InitialDeterminationDate_copy'],
    errors='coerce'
)

# -------------------------------------------------
# Keep latest record per Control_ID
# -------------------------------------------------
df_sorted = df_hum_drg_rem.sort_values(
    'InitialDeterminationDate_copy',
    ascending=False
)

df_hum_latest = df_sorted.drop_duplicates(
    subset='Control_ID',
    keep='first'
).copy()

print(df_hum_latest.shape)

# -------------------------------------------------
# Normalize string columns
# -------------------------------------------------
df_hum_latest.loc[:, 'ADRG'] = (
    df_hum_latest['ADRG'].astype(str).str.strip()
)

df_hum_latest.loc[:, 'PRIM_DX'] = (
    df_hum_latest['PRIM_DX'].astype(str).str.strip().str.upper()
)

df_hum_latest.loc[:, 'InitialDeterminationStatus'] = (
    df_hum_latest['InitialDeterminationStatus']
    .astype(str)
    .str.strip()
    .str.upper()
)

# -------------------------------------------------
# Clean numeric fields
# -------------------------------------------------
df_hum_latest.loc[:, 'IDSavings'] = (
    pd.to_numeric(df_hum_latest['IDSavings'], errors='coerce').fillna(0)
)

df_hum_latest.loc[:, 'LOS'] = (
    pd.to_numeric(df_hum_latest['LOS'], errors='coerce').fillna(0)
)

df_hum_latest.loc[:, 'AGE'] = (
    pd.to_numeric(df_hum_latest['AGE'], errors='coerce')
)

# -------------------------------------------------
# Validate AGE values
# -------------------------------------------------
max_age = df_hum_latest['AGE'].max()

df_hum_latest = df_hum_latest[
    df_hum_latest['AGE'].between(0, max_age)
].copy()

df_hum_latest.loc[:, 'AGE'] = df_hum_latest['AGE'].astype('Int64')
print(df_hum_latest.shape)

# -------------------------------------------------
# Validate LOS values
# -------------------------------------------------
df_hum_latest.loc[:, 'LOS'] = (
    df_hum_latest['LOS'].astype('Int64')
)

df_hum_latest = df_hum_latest[df_hum_latest['LOS'] >= 0].copy()
print(df_hum_latest.shape)

# -------------------------------------------------
# Remove extreme savings outliers
# -------------------------------------------------
df_hum_latest = df_hum_latest[
    df_hum_latest['IDSavings'] < 650000
].reset_index(drop=True)

print(df_hum_latest.shape)

# Date only
df_hum_latest['InitialDeterminationDate_date'] = (
    df_hum_latest['InitialDeterminationDate'].dt.date
)

# Year
df_hum_latest['InitialDetermination_Year'] = (
    df_hum_latest['InitialDeterminationDate'].dt.year
)

df_hum_latest["InitialDeterminationDate_date"] = pd.to_datetime(
    df_hum_latest["InitialDeterminationDate_date"],
    errors="coerce"   # handles bad / empty values safely
)

(73699, 291)
(73699, 291)
<StringArray>
['01', '10']
Length: 2, dtype: string
(73695, 291)
len_adrg
3    72660
4     1035
Name: count, dtype: int64
Group_Type
01    72660
10     1035
Name: count, dtype: Int64
(73687, 292)
Control_ID
HUPTMR-65573    1
HUPTMR-10204    1
HUPTMR-49129    1
HUPTMR-9989     1
HUPTMR-10906    1
Name: count, dtype: Int64
(73687, 293)
(73687, 293)
(73685, 293)
(73685, 293)


In [7]:
df_hum_ms_drg = df_hum_latest[df_hum_latest['Group_Type']=='01']
print(df_hum_ms_drg.shape)

df_hum_ms_drg['len_adrg'].value_counts()

(72654, 295)


len_adrg
3    72654
Name: count, dtype: int64

In [9]:
df = df_hum_ms_drg.copy()

adx_cols = [f"A_DX{i}" for i in range(2, 26)]
df["A_DX_List"] = df[adx_cols].values.tolist()
df["A_DX_List"] = df["A_DX_List"].apply(lambda x: sorted([i for i in x if pd.notna(i)]))

adx_cols = [f"B_DX{i}" for i in range(2, 26)]
df["B_DX_List"] = df[adx_cols].values.tolist()
df["B_DX_List"] = df["B_DX_List"].apply(lambda x: sorted([i for i in x if pd.notna(i)]))


def clean_sdx_list(lst):
    cleaned = []
    for item in lst:
        item = str(item).upper().strip()               # normalize
        item = item.replace("- MCC", "")               # remove MCC
        item = item.replace("- CC", "")                # remove CC
        item = item.replace("MCC", "")                 # safety
        item = item.replace("CC", "")                  # safety
        cleaned.append(item.strip().replace("-", ""))  # final cleanup
    return cleaned
df["A_DX_List_Clean"] = df["A_DX_List"].apply(clean_sdx_list)
df["B_DX_List_Clean"] = df["B_DX_List"].apply(clean_sdx_list)
 
# ---------------------------
# ------------------------------
# Step 1: Load MCC/CC Mapping File
# ---------------------------------------------------------
mccandcclist_df = pd.read_excel(r"C:\Arun_MIX\MCCCCList.xlsx")
mccandcclist_df['ICDCode'] = (
    mccandcclist_df['ICDCode']
    .astype(str)
    .str.strip()
    .str.upper()
)
mccandcclist = dict(zip(mccandcclist_df['ICDCode'], mccandcclist_df['MCCorCC']))
print(f"✅ Loaded MCC/CC list with {len(mccandcclist)} entries.")
mccandcclist_df.head()


def map_sdx_types(icd_list, lookup_dict):
    cleaned_list = []
    for code in icd_list:
        code_clean = str(code).strip().upper()  # normalize

        tag = lookup_dict.get(code_clean, "")  # MCC / CC / General
        cleaned_list.append(f"{code_clean} - {tag}")

    return cleaned_list

df['A_DX_Type_list'] = df['A_DX_List_Clean'].apply(lambda x: map_sdx_types(x, mccandcclist))
df['B_DX_Type_list'] = df['B_DX_List_Clean'].apply(lambda x: map_sdx_types(x, mccandcclist))

def extract_sdx_info(sdx_list):

    # If list is empty or missing
    if not sdx_list or len(sdx_list) == 0:
        return "No SDX Present", "None"

    # Separate entries by type
    mcc = [x.split(" - ")[0] for x in sdx_list if "- MCC" in x]
    cc  = [x.split(" - ")[0] for x in sdx_list if "- CC" in x]
    other = [
        x.split(" - ")[0]
        for x in sdx_list
        if "- MCC" not in x and "- CC" not in x
    ]

    # Determine SDX_Type and SDX_Set
    if mcc:
        return "MCC", ", ".join(mcc)
    elif cc:
        return "CC", ", ".join(cc)
    elif other:
        return "General", "None" #.join(other) #"No MCC or CC","None"
    else:
        return  "No SDX Present", "None"



# Apply the logic
df[["A_DX_Type", "A_DX_Set"]] = (
    df["A_DX_Type_list"]
    .apply(lambda x: pd.Series(extract_sdx_info(x)))
)

df[["B_DX_Type", "B_DX_Set"]] = (
    df["B_DX_Type_list"]
    .apply(lambda x: pd.Series(extract_sdx_info(x)))
)



pdx_cols = [f"A_PX{i}" for i in range(1, 26)]

# Step 1: Collect procedure columns into a list
df["A_Proc_List"] = df[pdx_cols].values.tolist()

# Step 2: Create comma-separated list or fill 'None' if empty
df["A_Proc_List"] = df["A_Proc_List"].apply(
    lambda x: (
        ",".join(map(str, sorted([i for i in x if pd.notna(i)])))
        if any(pd.notna(i) for i in x)
        else "None"
    )
)


pdx_cols = [f"B_PX{i}" for i in range(1, 26)]

# Step 1: Collect procedure columns into a list
df["B_Proc_List"] = df[pdx_cols].values.tolist()

# Step 2: Create comma-separated list or fill 'None' if empty
df["B_Proc_List"] = df["B_Proc_List"].apply(
    lambda x: (
        ",".join(map(str, sorted([i for i in x if pd.notna(i)])))
        if any(pd.notna(i) for i in x)
        else "None"
    )
)




✅ Loaded MCC/CC list with 17913 entries.


In [10]:
def clean_dx_code(x):
    if pd.isna(x):
        return ""
    
    # Convert to string and normalize
    x = str(x).upper().strip()
    
    # Remove square brackets if any
    x = x.replace("[", "").replace("]", "")
    
    # Remove MCC / CC suffixes
    x = x.replace("- MCC", "").replace("- CC", "")
    x = x.replace("MCC", "").replace("CC", "")
    
    # Remove leftover hyphens
    x = x.replace("-", "")
    
    return x
df["B_PRIMDX_Clean"] = df["B_PRIMDX"].apply(clean_dx_code)

In [17]:
df = df.copy()

df['InitialDeterminationStatus_Flag'] = (
    df['InitialDeterminationStatus']
    .str.strip()
    .str.upper()
    .map({
        'APPROVED': 0,
        'DENIED': 1
    })
)

In [81]:
required_cols = [
    "Payor","claimid","Client","Control_ID","Claim_Number","PaymentType",
    "Group_Type","Payments","Tax_ID","Provider_ID","HospitalName","LOS",
    "DISP_Code","ADRG","DischargeYear","DischargeMonth","AGE",
    "A_CCCount","A_MCCCount","B_CCCount","B_MCCCount","LOB",
    "MRREQ","MRREC","PRIM_DX",
    "B_DISPCode","B_DenialCode","B_DRG","B_PRIMDX",
    "InitialDeterminationDate","InitialDenialType",
    "InitialDeterminationStatus","IDSavings","AdjustedSavings",
    "DXRemoved_AtoB","PRIMDX_DownCoded",
    "InitialCoder","InitialCoder_Employer","InitialCoderID",
    "InitialDoctor","InitialDoctorID",
    "InitialProcessor","InitialProcessorID",
    "len_adrg","InitialDeterminationDate_copy",
    "InitialDeterminationDate_date","InitialDetermination_Year",
    "A_DX_List","B_DX_List",
    "A_DX_List_Clean","B_DX_List_Clean",
    "A_DX_Type_list","B_DX_Type_list",
    "A_DX_Type","A_DX_Set",
    "B_DX_Type","B_DX_Set",
    "A_Proc_List","B_Proc_List",
    "B_PRIMDX_Clean","InitialDeterminationStatus_Flag"
]

df_filtered = df[required_cols]
df_filtered.shape

(72654, 61)

In [83]:
df_filtered.to_excel("20260219 Humana Data.xlsx")

# Post Pay + Pre Pay

In [ ]:
from collections import defaultdict
import re

stats = defaultdict(lambda: {
    'Total_Claims': 0,
    'Approved': 0,
    'Denied': 0,
    'Total_Savings': 0.0
})

for idx, row in df.iterrows():

    drg = row['ADRG']
    pdx = row['PRIM_DX']
    saving = row['IDSavings']
    audit_result = row['InitialDeterminationStatus_Flag']
    status = 'APPROVED' if audit_result == 0 else 'DENIED'
  
    group_key = (drg,pdx)

    # --- Update stats ---
    stats[group_key]['Total_Claims'] += 1
   
    if status == 'APPROVED':
        stats[group_key]['Approved'] += 1
    else:
        stats[group_key]['Denied'] += 1
        if audit_result == 1:
            stats[group_key]['Total_Savings'] += float(saving)

# -----------------------------
# 7. Build Final DataFrame
# -----------------------------
final_data = []

for key, data in stats.items():
    drg,pdx = key   # ✅ FIXED unpacking

    total_claims = data['Total_Claims']
    approved = data['Approved']
    denied = data['Denied']
    total_savings = round(data['Total_Savings'], 2)
    approval_percent = round((approved / total_claims) * 100, 2) if total_claims else 0
    denial_percent = round((denied / total_claims) * 100, 2) if total_claims else 0
    avg_saving_per_claim = round(total_savings / total_claims, 2) if total_claims else 0
    avg_saving_per_den_claim = round(total_savings / denied, 2) if denied else 0



    final_data.append({
        'DRG': drg,
        'PDX': pdx,
        'Total_Claims': total_claims,
        'Approved_Claims': approved,
        'Approval_Percent': approval_percent,
        'Denied_Claims': denied,
        'Denial_Percent': denial_percent,
        'Total_Savings': total_savings,
        'Avg_Saving_per_Claim': avg_saving_per_claim,
        'Avg_Saving_per_Denied_Claim': avg_saving_per_den_claim

    })

df_com = pd.DataFrame(final_data)

df_com['Referral_%'] = (((30 * df_com['Denial_Percent'])/70) + df_com['Denial_Percent'])/100
df_com['Phy_Cost'] = df_com['Referral_%'] * df_com['Total_Claims'] * 50
df_com['Coder_Cost'] = df_com['Total_Claims'] * 12
df_com['Expense'] = df_com['Phy_Cost'] + df_com['Coder_Cost']
df_com['Expense_Per_Claim'] = df_com['Expense'] / df_com['Total_Claims']

df_com['Commission'] = df_com['Total_Savings'] * 0.17
df_com['Commission_Per_Claim'] = df_com['Commission'] / df_com['Total_Claims']

df_com['Net_Profit'] = df_com['Commission'] - df_com['Expense']
df_com['Net_Profit_Per_Claim'] = df_com['Net_Profit']/ df_com['Total_Claims']

df_com.to_excel("ALL Humnana DRG+PDX Denial Analysis.xlsx")

required_columns = [
    'DRG',
    'PDX',
    'Total_Claims',
    'Denied_Claims',
    'Denial_Percent',
    'Total_Savings',
    'Avg_Saving_per_Claim',
    'Expense_Per_Claim',
    'Commission_Per_Claim',
    'Net_Profit_Per_Claim'
]

df_com = df_com[required_columns]
df_com.head()


# Convert required columns to numeric
numeric_cols = [
    'Denial_Percent',
    'Avg_Saving_per_Claim',
    'Total_Claims'
]

for col in numeric_cols:
    df_com[col] = pd.to_numeric(df_com[col], errors='coerce')

# Base condition
base_condition = df_com['Total_Claims'] >= 5

# ---------------------------
# 4 Combination DataFrames
# ---------------------------

HighDen_HighSaving = df_com[
    base_condition &
    (df_com['Denial_Percent'] >= 30) &
    (df_com['Avg_Saving_per_Claim'] >= 1000)
]

LowDen_HighSaving = df_com[
    base_condition &
    (df_com['Denial_Percent'] < 30) &
    (df_com['Avg_Saving_per_Claim'] >= 1000)
]

HighDen_MidSaving = df_com[
    base_condition &
    (df_com['Denial_Percent'] >= 30) &
    (df_com['Avg_Saving_per_Claim'] >= 250) &
    (df_com['Avg_Saving_per_Claim'] < 1000)
]

LowDen_MidSaving = df_com[
    base_condition &
    (df_com['Denial_Percent'] < 30) &
    (df_com['Avg_Saving_per_Claim'] >= 250) &
    (df_com['Avg_Saving_per_Claim'] < 1000)
]

LowDen_LowSaving = df_com[
    base_condition &
    (df_com['Denial_Percent'] < 30) &
    (df_com['Avg_Saving_per_Claim'] < 250)
]



# ---------------------------
# Remaining Claim Details (> 5 Claims)
# ---------------------------

Remaining_Claim_Details = df_com[
    df_com['Total_Claims'] < 5
]

# Function to calculate summary
def create_summary(df, label):
    
    unique_combo = df[['DRG', 'PDX']].drop_duplicates().shape[0]
    total_claims = df['Total_Claims'].sum()
    denied_claims = df['Denied_Claims'].sum()
    denial_percent = (denied_claims / total_claims * 100) if total_claims != 0 else 0
    total_saving = df['Total_Savings'].sum()
    
    return {
        'Category': label,
        'Unique Combination (DRG+PDX)': unique_combo,
        'Claims': total_claims,
        'Denied Claims': denied_claims,
        'Denied %': round(denial_percent, 2),
        'Total Saving': round(total_saving, 2)
    }


# Create Summary Data
summary_data = [
    create_summary(HighDen_HighSaving, "High Den % & High Saving per Claim"),
    create_summary(LowDen_HighSaving, "Low Den % & High Saving per Claim"),
    create_summary(HighDen_MidSaving, "High Den % & Saving per Claim B/W $250 to $1000"),
    create_summary(LowDen_MidSaving, "Low Den % & Saving per Claim B/W $250 to $1000"),
    create_summary(LowDen_LowSaving, "Low Den % & Low Saving per Claim"),
    create_summary(Remaining_Claim_Details, "Remainung Claims < 5")
]

# Convert to DataFrame
df_summary = pd.DataFrame(summary_data)

df_summary.head()


# ---------------------------
# Export to Excel
# ---------------------------

with pd.ExcelWriter("DRG_PDX_Combination_Denial_Analysis.xlsx", engine="openpyxl") as writer:
    df_summary.to_excel(writer, sheet_name="Insights", index=False)
    HighDen_HighSaving.to_excel(writer, sheet_name="HighDen_HighSaving", index=False)
    LowDen_HighSaving.to_excel(writer, sheet_name="LowDen_HighSaving", index=False)
    HighDen_MidSaving.to_excel(writer, sheet_name="HighDen_MidSaving", index=False)
    LowDen_MidSaving.to_excel(writer, sheet_name="LowDen_MidSaving", index=False)
    LowDen_LowSaving.to_excel(writer, sheet_name="LowDen_LowSaving", index=False)
    Remaining_Claim_Details.to_excel(writer, sheet_name="Remaining_Claims_<5", index=False)

print("Excel file created successfully ✅")


# Post pay

In [68]:
df['PaymentType'].value_counts()

PaymentType
PostPay    64003
Prepay      8651
Name: count, dtype: Int64

In [69]:
df_ms_drg_postpay =df[df['PaymentType'] == 'PostPay']
df_ms_drg_postpay.shape

(64003, 309)

In [ ]:
from collections import defaultdict
import re

stats = defaultdict(lambda: {
    'Total_Claims': 0,
    'Approved': 0,
    'Denied': 0,
    'Total_Savings': 0.0
})

for idx, row in df_ms_drg_postpay.iterrows():

    drg = row['ADRG']
    pdx = row['PRIM_DX']
    saving = row['IDSavings']
    audit_result = row['InitialDeterminationStatus_Flag']
    status = 'APPROVED' if audit_result == 0 else 'DENIED'
  
    group_key = (drg,pdx)

    # --- Update stats ---
    stats[group_key]['Total_Claims'] += 1
   
    if status == 'APPROVED':
        stats[group_key]['Approved'] += 1
    else:
        stats[group_key]['Denied'] += 1
        if audit_result == 1:
            stats[group_key]['Total_Savings'] += float(saving)

# -----------------------------
# 7. Build Final DataFrame
# -----------------------------
final_data = []

for key, data in stats.items():
    drg,pdx = key   # ✅ FIXED unpacking

    total_claims = data['Total_Claims']
    approved = data['Approved']
    denied = data['Denied']
    total_savings = round(data['Total_Savings'], 2)
    approval_percent = round((approved / total_claims) * 100, 2) if total_claims else 0
    denial_percent = round((denied / total_claims) * 100, 2) if total_claims else 0
    avg_saving_per_claim = round(total_savings / total_claims, 2) if total_claims else 0
    avg_saving_per_den_claim = round(total_savings / denied, 2) if denied else 0



    final_data.append({
        'DRG': drg,
        'PDX': pdx,
        'Total_Claims': total_claims,
        'Approved_Claims': approved,
        'Approval_Percent': approval_percent,
        'Denied_Claims': denied,
        'Denial_Percent': denial_percent,
        'Total_Savings': total_savings,
        'Avg_Saving_per_Claim': avg_saving_per_claim,
        'Avg_Saving_per_Denied_Claim': avg_saving_per_den_claim

    })

df_com = pd.DataFrame(final_data)

df_com['Referral_%'] = (((30 * df_com['Denial_Percent'])/70) + df_com['Denial_Percent'])/100
df_com['Phy_Cost'] = df_com['Referral_%'] * df_com['Total_Claims'] * 50
df_com['Coder_Cost'] = df_com['Total_Claims'] * 12
df_com['Expense'] = df_com['Phy_Cost'] + df_com['Coder_Cost']
df_com['Expense_Per_Claim'] = df_com['Expense'] / df_com['Total_Claims']

df_com['Commission'] = df_com['Total_Savings'] * 0.17
df_com['Commission_Per_Claim'] = df_com['Commission'] / df_com['Total_Claims']

df_com['Net_Profit'] = df_com['Commission'] - df_com['Expense']
df_com['Net_Profit_Per_Claim'] = df_com['Net_Profit']/ df_com['Total_Claims']

df_com.head()


required_columns = [
    'DRG',
    'PDX',
    'Total_Claims',
    'Denied_Claims',
    'Denial_Percent',
    'Total_Savings',
    'Avg_Saving_per_Claim',
    'Expense_Per_Claim',
    'Commission_Per_Claim',
    'Net_Profit_Per_Claim'
]

df_com = df_com[required_columns]
df_com.head()

df_com.to_excel("PostPay Humnana DRG+PDX Denial Analysis.xlsx")



# Convert required columns to numeric
numeric_cols = [
    'Denial_Percent',
    'Avg_Saving_per_Claim',
    'Total_Claims'
]

for col in numeric_cols:
    df_com[col] = pd.to_numeric(df_com[col], errors='coerce')

# Base condition
base_condition = df_com['Total_Claims'] >= 5

# ---------------------------
# 4 Combination DataFrames
# ---------------------------

post_HighDen_HighSaving = df_com[
    base_condition &
    (df_com['Denial_Percent'] >= 30) &
    (df_com['Avg_Saving_per_Claim'] >= 1000)
]

post_LowDen_HighSaving = df_com[
    base_condition &
    (df_com['Denial_Percent'] < 30) &
    (df_com['Avg_Saving_per_Claim'] >= 1000)
]

post_HighDen_MidSaving = df_com[
    base_condition &
    (df_com['Denial_Percent'] >= 30) &
    (df_com['Avg_Saving_per_Claim'] >= 250) &
    (df_com['Avg_Saving_per_Claim'] < 1000)
]

post_LowDen_MidSaving = df_com[
    base_condition &
    (df_com['Denial_Percent'] < 30) &
    (df_com['Avg_Saving_per_Claim'] >= 250) &
    (df_com['Avg_Saving_per_Claim'] < 1000)
]

post_LowDen_LowSaving = df_com[
    base_condition &
    (df_com['Denial_Percent'] < 30) &
    (df_com['Avg_Saving_per_Claim'] < 250)
]

# ---------------------------
# Remaining Claim Details (> 5 Claims)
# ---------------------------

post_Remaining_Claim_Details = df_com[
    df_com['Total_Claims'] < 5
]

# Function to calculate summary
def create_summary(df, label):
    
    unique_combo = df[['DRG', 'PDX']].drop_duplicates().shape[0]
    total_claims = df['Total_Claims'].sum()
    denied_claims = df['Denied_Claims'].sum()
    denial_percent = (denied_claims / total_claims * 100) if total_claims != 0 else 0
    total_saving = df['Total_Savings'].sum()
    
    return {
        'Category': label,
        'Unique Combination (DRG+PDX)': unique_combo,
        'Claims': total_claims,
        'Denied Claims': denied_claims,
        'Denied %': round(denial_percent, 2),
        'Total Saving': round(total_saving, 2)
    }


# Create Summary Data
summary_data = [
    create_summary(post_HighDen_HighSaving, "High Den % & High Saving per Claim"),
    create_summary(post_LowDen_HighSaving, "Low Den % & High Saving per Claim"),
    create_summary(post_HighDen_MidSaving, "High Den % & Saving per Claim B/W $250 to $1000"),
    create_summary(post_LowDen_MidSaving, "Low Den % & Saving per Claim B/W $250 to $1000"),
    create_summary(post_LowDen_LowSaving, "Low Den % & Low Saving per Claim")
    create_summary(post_Remaining_Claim_Details, "Remainung Claims < 5")
]

# Convert to DataFrame
df_post_summary = pd.DataFrame(summary_data)

df_post_summary.head()


# ---------------------------
# Export to Excel
# ---------------------------

with pd.ExcelWriter("Postpay DRG_PDX_Combination_Analysis 2024 2025.xlsx", engine="openpyxl") as writer:
    df_post_summary.to_excel(writer, sheet_name="post_Insights", index=False)
    post_HighDen_HighSaving.to_excel(writer, sheet_name="post_HighDen_HighSaving", index=False)
    post_LowDen_HighSaving.to_excel(writer, sheet_name="post_LowDen_HighSaving", index=False)
    post_HighDen_MidSaving.to_excel(writer, sheet_name="post_HighDen_MidSaving", index=False)
    post_LowDen_MidSaving.to_excel(writer, sheet_name="post_LowDen_MidSaving", index=False)
    post_LowDen_LowSaving.to_excel(writer, sheet_name="post_LowDen_LowSaving", index=False)
    post_Remaining_Claim_Details.to_excel(writer, sheet_name="post_Remaining_Claims_<5", index=False)

print("Excel file created successfully ✅")





Excel file created successfully ✅


# prepay

In [72]:
df_ms_drg_prepay =df[df['PaymentType'] == 'Prepay']
df_ms_drg_prepay.shape

(8651, 309)

In [ ]:
from collections import defaultdict
import re

stats = defaultdict(lambda: {
    'Total_Claims': 0,
    'Approved': 0,
    'Denied': 0,
    'Total_Savings': 0.0
})

for idx, row in df_ms_drg_prepay.iterrows():

    drg = row['ADRG']
    pdx = row['PRIM_DX']
    saving = row['IDSavings']
    audit_result = row['InitialDeterminationStatus_Flag']
    status = 'APPROVED' if audit_result == 0 else 'DENIED'
  
    group_key = (drg,pdx)

    # --- Update stats ---
    stats[group_key]['Total_Claims'] += 1
   
    if status == 'APPROVED':
        stats[group_key]['Approved'] += 1
    else:
        stats[group_key]['Denied'] += 1
        if audit_result == 1:
            stats[group_key]['Total_Savings'] += float(saving)

# -----------------------------
# 7. Build Final DataFrame
# -----------------------------
final_data = []

for key, data in stats.items():
    drg,pdx = key   # ✅ FIXED unpacking

    total_claims = data['Total_Claims']
    approved = data['Approved']
    denied = data['Denied']
    total_savings = round(data['Total_Savings'], 2)
    approval_percent = round((approved / total_claims) * 100, 2) if total_claims else 0
    denial_percent = round((denied / total_claims) * 100, 2) if total_claims else 0
    avg_saving_per_claim = round(total_savings / total_claims, 2) if total_claims else 0
    avg_saving_per_den_claim = round(total_savings / denied, 2) if denied else 0



    final_data.append({
        'DRG': drg,
        'PDX': pdx,
        'Total_Claims': total_claims,
        'Approved_Claims': approved,
        'Approval_Percent': approval_percent,
        'Denied_Claims': denied,
        'Denial_Percent': denial_percent,
        'Total_Savings': total_savings,
        'Avg_Saving_per_Claim': avg_saving_per_claim,
        'Avg_Saving_per_Denied_Claim': avg_saving_per_den_claim

    })

df_com = pd.DataFrame(final_data)

df_com['Referral_%'] = (((30 * df_com['Denial_Percent'])/70) + df_com['Denial_Percent'])/100
df_com['Phy_Cost'] = df_com['Referral_%'] * df_com['Total_Claims'] * 50
df_com['Coder_Cost'] = df_com['Total_Claims'] * 12
df_com['Expense'] = df_com['Phy_Cost'] + df_com['Coder_Cost']
df_com['Expense_Per_Claim'] = df_com['Expense'] / df_com['Total_Claims']

df_com['Commission'] = df_com['Total_Savings'] * 0.17
df_com['Commission_Per_Claim'] = df_com['Commission'] / df_com['Total_Claims']

df_com['Net_Profit'] = df_com['Commission'] - df_com['Expense']
df_com['Net_Profit_Per_Claim'] = df_com['Net_Profit']/ df_com['Total_Claims']

df_com.head()


required_columns = [
    'DRG',
    'PDX',
    'Total_Claims',
    'Denied_Claims',
    'Denial_Percent',
    'Total_Savings',
    'Avg_Saving_per_Claim',
    'Expense_Per_Claim',
    'Commission_Per_Claim',
    'Net_Profit_Per_Claim'
]

df_com = df_com[required_columns]
df_com.head()


df_com.to_excel("PrePay Humnana DRG+PDX Denial Analysis.xlsx")

# Convert required columns to numeric
numeric_cols = [
    'Denial_Percent',
    'Avg_Saving_per_Claim',
    'Total_Claims'
]

for col in numeric_cols:
    df_com[col] = pd.to_numeric(df_com[col], errors='coerce')

# Base condition
base_condition = df_com['Total_Claims'] >= 5

# ---------------------------
# 4 Combination DataFrames
# ---------------------------

pre_HighDen_HighSaving = df_com[
    base_condition &
    (df_com['Denial_Percent'] >= 30) &
    (df_com['Avg_Saving_per_Claim'] >= 1000)
]

pre_LowDen_HighSaving = df_com[
    base_condition &
    (df_com['Denial_Percent'] < 30) &
    (df_com['Avg_Saving_per_Claim'] >= 1000)
]

pre_HighDen_MidSaving = df_com[
    base_condition &
    (df_com['Denial_Percent'] >= 30) &
    (df_com['Avg_Saving_per_Claim'] >= 250) &
    (df_com['Avg_Saving_per_Claim'] < 1000)
]

pre_LowDen_MidSaving = df_com[
    base_condition &
    (df_com['Denial_Percent'] < 30) &
    (df_com['Avg_Saving_per_Claim'] >= 250) &
    (df_com['Avg_Saving_per_Claim'] < 1000)
]

pre_LowDen_LowSaving = df_com[
    base_condition &
    (df_com['Denial_Percent'] < 30) &
    (df_com['Avg_Saving_per_Claim'] < 250)
]

# ---------------------------
# Remaining Claim Details (> 5 Claims)
# ---------------------------

pre_Remaining_Claim_Details = df_com[
    df_com['Total_Claims'] < 5
]

# Function to calculate summary
def create_summary(df, label):
    
    unique_combo = df[['DRG', 'PDX']].drop_duplicates().shape[0]
    total_claims = df['Total_Claims'].sum()
    denied_claims = df['Denied_Claims'].sum()
    denial_percent = (denied_claims / total_claims * 100) if total_claims != 0 else 0
    total_saving = df['Total_Savings'].sum()
    
    return {
        'Category': label,
        'Unique Combination (DRG+PDX)': unique_combo,
        'Claims': total_claims,
        'Denied Claims': denied_claims,
        'Denied %': round(denial_percent, 2),
        'Total Saving': round(total_saving, 2)
    }


# Create Summary Data
summary_data = [
    create_summary(pre_HighDen_HighSaving, "High Den % & High Saving per Claim"),
    create_summary(pre_LowDen_HighSaving, "Low Den % & High Saving per Claim"),
    create_summary(pre_HighDen_MidSaving, "High Den % & Saving per Claim B/W $250 to $1000"),
    create_summary(pre_LowDen_MidSaving, "Low Den % & Saving per Claim B/W $250 to $1000"),
    create_summary(pre_LowDen_LowSaving, "Low Den % & Low Saving per Claim"),
    create_summary(pre_Remaining_Claim_Details, "pre_Remaining_Claims_<5")
]

# Convert to DataFrame
df_pre_summary = pd.DataFrame(summary_data)

df_pre_summary.head()


# ---------------------------
# Export to Excel
# ---------------------------

with pd.ExcelWriter("Prepay DRG_PDX_Combination_Analysis 2024 2025.xlsx", engine="openpyxl") as writer:
    df_pre_summary.to_excel(writer, sheet_name="pre_Insights", index=False)
    pre_HighDen_HighSaving.to_excel(writer, sheet_name="pre_HighDen_HighSaving", index=False)
    pre_LowDen_HighSaving.to_excel(writer, sheet_name="pre_LowDen_HighSaving", index=False)
    pre_HighDen_MidSaving.to_excel(writer, sheet_name="pre_HighDen_MidSaving", index=False)
    pre_LowDen_MidSaving.to_excel(writer, sheet_name="pre_LowDen_MidSaving", index=False)
    pre_LowDen_LowSaving.to_excel(writer, sheet_name="pre_LowDen_LowSaving", index=False)
    pre_Remaining_Claim_Details.to_excel(writer, sheet_name="pre_Remaining_Claims_<5", index=False)

print("Excel file created successfully ✅")





Excel file created successfully ✅


# all

In [ ]:
with pd.ExcelWriter("DRG_PDX_Combination_Analysis 2024 2025.xlsx", engine="openpyxl") as writer:
    df_summary.to_excel(writer, sheet_name="Insights", index=False)
    HighDen_HighSaving.to_excel(writer, sheet_name="HighDen_HighSaving", index=False)
    LowDen_HighSaving.to_excel(writer, sheet_name="LowDen_HighSaving", index=False)
    HighDen_MidSaving.to_excel(writer, sheet_name="HighDen_MidSaving", index=False)
    LowDen_MidSaving.to_excel(writer, sheet_name="LowDen_MidSaving", index=False)
    LowDen_LowSaving.to_excel(writer, sheet_name="LowDen_LowSaving", index=False)
    Remaining_Claim_Details.to_excel(writer, sheet_name="Remaining_Claims_<5", index=False)

    df_post_summary.to_excel(writer, sheet_name="post_Insights", index=False)
    post_HighDen_HighSaving.to_excel(writer, sheet_name="post_HighDen_HighSaving", index=False)
    post_LowDen_HighSaving.to_excel(writer, sheet_name="post_LowDen_HighSaving", index=False)
    post_HighDen_MidSaving.to_excel(writer, sheet_name="post_HighDen_MidSaving", index=False)
    post_LowDen_MidSaving.to_excel(writer, sheet_name="post_LowDen_MidSaving", index=False)
    post_LowDen_LowSaving.to_excel(writer, sheet_name="post_LowDen_LowSaving", index=False)
    post_Remaining_Claim_Details.to_excel(writer, sheet_name="post_Remaining_Claims_<5", index=False)

    df_pre_summary.to_excel(writer, sheet_name="pre_Insights", index=False)
    pre_HighDen_HighSaving.to_excel(writer, sheet_name="pre_HighDen_HighSaving", index=False)
    pre_LowDen_HighSaving.to_excel(writer, sheet_name="pre_LowDen_HighSaving", index=False)
    pre_HighDen_MidSaving.to_excel(writer, sheet_name="pre_HighDen_MidSaving", index=False)
    pre_LowDen_MidSaving.to_excel(writer, sheet_name="pre_LowDen_MidSaving", index=False)
    pre_LowDen_LowSaving.to_excel(writer, sheet_name="pre_LowDen_LowSaving", index=False)
    pre_Remaining_Claim_Details.to_excel(writer, sheet_name="pre_Remaining_Claims_<5", index=False)

    